In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage
from mesostat.utils.pandas_helper import merge_df_from_dict

from lib.sych.data_fc_db_raw import DataFCDatabase

import lib.analysis.pid as pid

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
#params['root_path_data'] = gui_fpath('h5path', './')

In [4]:
dataDB = DataFCDatabase(params)

Searching for data files
Extracting trial type names
Extracting data types


In [5]:
dataDB.mice

{'mvg_4', 'mvg_7', 'mvg_8', 'mvg_9'}

In [ ]:
h5outname = 'sych_result_higher_order_df.h5'

In [ ]:
mc = MetricCalculator(serial=False, verbose=False, nCore=4)

# TODO
    
Hypothesis - Chain inhibition should increase synergy
* Cpu -> iGP/GP/eGP -> VM/VL

Performance-depencence
* Session-wise changes of redundancy/synergy as function of performance
* Movement-correlations of synergy/redundancy
    - Lick
    - Integral movement

# Hypotheses

In [ ]:
interv_dict = {
    "TEX" : [3, 3.5],
    "REW" : [6, 6.5]
}

hypotheses_dict = {
    # Feedforwards Prefrontal
    "H1_TEX"  : ("TEX", ['M1_l', 'S1_bf'], ['PrL', 'LO', 'VO', 'M2', 'Cg1']),
    "H1a_TEX" : ("TEX", ['S1_bf', 'S2'],   ['PrL', 'LO', 'VO', 'M2', 'Cg1']),
    "H1b_TEX" : ("TEX", ['M1_l', 'M2'],    ['PrL', 'LO', 'VO', 'Cg1']),  # Drop M2 because its a source

    # High order is Sensory/Motor Thalamus
    # Test if (M1, S1) has more synergy than (M1, M2) or (S1, S2)
    "H2_TEX"  : ("TEX", ['M1_l', 'S1_bf'], ['Po', 'VM']),
    "H2a_TEX" : ("TEX", ['S1_bf', 'S2'],   ['Po', 'VM']),
    "H2b_TEX" : ("TEX", ['M1_l', 'M2'],    ['Po', 'VM']),
    
    # Thalamus as source
    "H3_TEX"  : ("TEX", ['Po', 'VPM'],     ['S1_bf', 'S2']),
    
    # Motor Thalamus synchronization
    "H4_TEX"  : ("TEX", ['VM', 'VL'],      ['M1_l', 'M2']),
    "H4a_TEX" : ("TEX", ['VM', 'LDVL'],    ['M1_l', 'M2']),
    "H4b_TEX" : ("TEX", ['VL', 'LDVL'],    ['M1_l', 'M2']),
}

In [ ]:
for datatype in ['bn_session', 'bn_trial']:
    for hLabel, (intervKey, sources, targets) in hypotheses_dict.items():
        print(hLabel)

        interv = interv_dict[intervKey]

        # Calculate PID
        rezDict = {}
        for mousename in dataDB.mice:
            channelNames = dataDB.get_channel_labels(mousename)
            dataLst = dataDB.get_neuro_data({'mousename': mousename}, datatype=datatype,
                                            zscoreDim=None, cropTime=interv,
                                            trialType='iGO', performance='expert')
            
            rezDict[(mousename,)] = pid.pid(dataLst, mc, channelNames, sources, targets, nPerm=2000, nBin=4)
            
        rezDF = merge_df_from_dict(rezDict, ['mousename'])

    #     # Display resulting dataframe
    #     with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #         display(rezDF.sort_values(by=['S1', 'S2', 'T', 'PID', 'mousename']))

        # Save to file
        dataLabel = '_'.join(['PID', datatype, hLabel])
        rezDF.to_hdf(h5outname, dataLabel, mode='a', format='table', data_columns=True)

In [ ]:
for datatype in ['bn_trial']:
    for hLabel, (intervKey, sources, targets) in hypotheses_dict.items():
        print(hLabel)

        dataLabel = '_'.join(['PID', datatype, hLabel])
        pid.plot_pval_aggr(h5outname, dataLabel)
        pid.plot_metric_bymouse(h5outname, dataLabel, 'muTrue', 'Info(Bits)', yscale='log', clip=[1.0E-3, 1], ylim=[1.0E-3, 1])

        interv = interv_dict[intervKey]
        for mousename in dataDB.mice:
            
            datakwargs = {
                'datatype' : datatype,
                'cropTime' : interv,
                'trialType' : 'iGO',
                'performance' : 'expert'
            }
            
            pid.info3D_plot_slice_trg(dataDB, sources, targets, datakwargs)

## Suppl

In [ ]:
np.array(dataDB.get_channel_labels('mvg_4'))